https://mgimond.github.io/Spatial/spatial-autocorrelation-in-r.html

In [ ]:
library(tmap)
load(url("http://github.com/mgimond/Spatial/raw/master/Data/moransI.RData"))
tm_shape(s1) + tm_polygons(style="quantile", col = "Income") +
  tm_legend(outside = TRUE, text.size = .8)

## Spatial weights of polygons
Spatial weights can be defined in two ways:
* Distance based weights - defined by distance between centroids
* Contiguity based weights - defined by connected neighbors



In [ ]:
library(spdep)
nb <- poly2nb(s1, queen=TRUE)
lw <- nb2listw(nb, style="W", zero.policy=TRUE)
lw$weights[1]

## Moran's I from regression

In [ ]:
library(spdep)
nb <- poly2nb(s1, queen=TRUE)
lw <- nb2listw(nb, style="W", zero.policy=TRUE)
Inc.lag <- lag.listw(lw, s1$Income)
M <- lm(Inc.lag ~ s1$Income)
plot( Inc.lag ~ s1$Income, pch=20, asp=1, las=1)
coef(M)[2]

In [ ]:
n <- 599L   # Define the number of simulations
I.r <- vector(length=n)  # Create an empty vector

for (i in 1:n){
  # Randomly shuffle income values
  x <- sample(s1$Income, replace=FALSE)
  # Compute new set of lagged values
  x.lag <- lag.listw(lw, x)
  # Compute the regression slope and store its value
  M.r    <- lm(x.lag ~ x)
  I.r[i] <- coef(M.r)[2]
}
# Plot the histogram of simulated Moran's I values
# then add our observed Moran's I value to the plot
hist(I.r, main=NULL, xlab="Moran's I", las=1)
abline(v=coef(M)[2], col="red")

In [ ]:
N.greater <- sum(coef(M)[2] > I.r)
p <- min(N.greater + 1, n + 1 - N.greater) / (n + 1)
p

In [ ]:
moran.test(s1$Income,lw)

## Use distance band as spatial weights

In [ ]:
coo <- coordinates(s1)
S.dist  <-  dnearneigh(coo, 0, 50000)  
lw <- nb2listw(S.dist, style="W",zero.policy=T)
MI  <-  moran.mc(s1$Income, lw, nsim=599,zero.policy=T)
plot(MI, main="", las=1) 
MI